# --- Day 6: Custom Customs ---

## Imports

In [1]:
import tensorflow as tf
try:
    # Disable all GPUS
    tf.config.set_visible_devices([], 'GPU')
    visible_devices = tf.config.get_visible_devices()
    for device in visible_devices:
        assert device.device_type != 'GPU'
except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass

# Fixes bad convolution
#gpus = tf.config.experimental.list_physical_devices('GPU')
#for gpu in gpus:
#    tf.config.experimental.set_memory_growth(gpu, True)

## Part 1

In [2]:
def count_yes_answers(x):
    x = x[tf.math.count_nonzero(x, axis=1)>0]
    x_flat = tf.reshape(x, (-1,))
    x_flat = x_flat[x_flat != 0]
    y, _, count = tf.unique_with_counts(x_flat)
    return tf.math.count_nonzero(count == tf.shape(x)[0])

@tf.function
def solve(s, part=1):
    s = tf.strings.split(s, '\n\n')
    s = tf.strings.split(s, '\n')
    s = tf.strings.unicode_decode(s, 'UTF-8')
    s = s.to_tensor()

    if part == 1:
        return tf.foldr(lambda acc, x: acc + tf.shape(tf.unique(tf.reshape(x, (-1,)))[0])[0]-1, s, 0)
    elif part == 2:
        return tf.foldr(lambda acc, x: acc + count_yes_answers(x), s, tf.constant(0, tf.int64))

solve(tf.io.read_file('day6.txt', 'r'), part=2)

<tf.Tensor: shape=(), dtype=int64, numpy=3290>

## Part 1

In [3]:
%%timeit
solve(tf.io.read_file('day6.txt', 'r'), part=1)

14.9 ms ± 155 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Part 2

In [4]:
%%timeit
solve(tf.io.read_file('day6.txt', 'r'), part=2)

32.6 ms ± 1.24 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [194]:
import datetime
# Set up logging.
stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = 'logs/day6_graph/%s' % stamp

# Bracket the function call with
# tf.summary.trace_on() and tf.summary.trace_export().
tf.profiler.experimental.start(logdir)
solve(tf.io.read_file('day6.txt', 'r'), part=2)
tf.profiler.experimental.stop()